In [1]:
from functions_scraping import select_pages, crawl_charts, create_file

In [7]:
years = list(range(1996, 2001))

for year in years:
    # select pages to crawl
    start = str(year-1) + '-12-31'
    end = str(year) + '-12-31'
    pages = select_pages(start, end)
    
    # web scraping of https://www.offiziellecharts.de for chosen dates
    charts = crawl_charts(pages)
    
    # create file for year
    create_file(charts, str(year)+'_top_100')

Request:53; Frequency: 0.08549112779116107 requests/s
